# Face Recognition with Siamese Network Model

## Importing necessary libraries and dependencies

In [60]:
import numpy as np
import os
from matplotlib import pyplot as plt
import cv2
from tqdm.notebook import tqdm, trange

In [3]:
#Importing tensorflow functional API dependencies

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

## Setting paths and generating dataset

In [4]:
#Setup paths
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

In [7]:
#Make directories
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

In [8]:
#Uncompress Tar GZ labelled Faces in the Wild Dataset
!tar -xf lfw.tgz

In [9]:
for directory in os.listdir('lfw'):
  for filE in os.listdir(os.path.join('lfw', directory)):
    EX_PATH = os.path.join('lfw', directory, filE)
    NEW_PATH = os.path.join(NEG_PATH, filE)
    os.replace(EX_PATH, NEW_PATH)

In [4]:
import uuid

#### Capturing images from webcam to be used as anchor and positive images

In [18]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
  ret, frame = cap.read()

  frame = frame[120:120+250, 200:200+250, :]

  if cv2.waitKey(1) & 0XFF == ord('a'):
    imgname = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
    cv2.imwrite(imgname, frame)

  if cv2.waitKey(1) & 0XFF == ord('p'):
    imgname = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
    cv2.imwrite(imgname, frame)

  cv2.imshow('Image Collection', frame)

  if cv2.waitKey(1) & 0XFF == ord('q'):
    break
cap.release()
cv2.destroyAllWindows()

In [19]:
len(os.listdir(POS_PATH))

311

In [5]:
anchor = tf.data.Dataset.list_files(ANC_PATH+'\*.jpg').take(300)
positive = tf.data.Dataset.list_files(POS_PATH+'\*.jpg').take(300)
negative = tf.data.Dataset.list_files(NEG_PATH+'\*.jpg').take(300)

#### Creating the preprocessing function

In [6]:
def preprocess(file_path):
    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(byte_img)
    img = tf.image.resize(img, (100,100))
    img = img/255.0
    return img

In [7]:
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)

In [40]:
samples = data.as_numpy_iterator()

In [37]:
ex = samples.next()

#### Building the train & test datasets

In [8]:
def update_prepdata(input_img, validation_img, label):
    return ((preprocess(input_img), preprocess(validation_img)), label)

In [9]:
data = data.map(update_prepdata)
data = data.cache()
data = data.shuffle(buffer_size=1024)

In [10]:
train_data = data.take(round(len(data)*0.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

In [11]:
test_data = data.skip(round(len(data)*0.7), )
test_data = test_data.take(round(len(data)*0.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

## Building the Siamese Network

#### The embedding layer

The anchor and validation images are passed through the embedding layer in order to obtain vectors containing encodings which represent the images

In [12]:
def create_embedding():

    inp = Input((100, 100, 3))

    #Block 1
    c1  = Conv2D(64, (10,10), activation='relu')(inp)
    m1 = MaxPooling2D((2,2), padding='same')(c1) #Stride isn't 1 it depends on pool_size

    #Block 2
    c2 = Conv2D(128, (7,7), activation='relu')(m1)
    m2 = MaxPooling2D((2,2), padding='same')(c2)

    #Block 3
    c3 = Conv2D(128, (4,4), activation='relu')(m2)
    m3 = MaxPooling2D((2,2), padding='same')(c3)

    #Final Block
    c4 = Conv2D(256, (4,4), activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid')(f1)

    return Model(inputs=[inp], outputs=[d1], name='embedding')

In [33]:
embedding = create_embedding()

In [34]:
embedding.summary()

Model: "embedding"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 conv2d_8 (Conv2D)           (None, 91, 91, 64)        19264     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 46, 46, 64)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 40, 40, 128)       401536    
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 20, 20, 128)      0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 17, 17, 128)       26

#### Creating a custom layer to combine the anchor and validation embeddings

In [35]:
class L1Dist(Layer):
    def __init__(self, **kwargs):
        super().__init__()
    
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding-validation_embedding)

#### Implementing the final block 

This block makes use of the embedding model made previously to obtain the feature embeddings of the images. It then combines the embeddings by passing them through the L1Dist layer. Finally, it passes the output of the L1Dist layer to a single neuron with sigmoid activation to generate predictions. 

In [36]:
def make_siamese():

    #anchor image
    inp_image = Input((100,100,3), name='input_image')

    #validation image
    val_image = Input((100,100,3), name='val_image')

    #Combine embeddings 
    siamese_layer = L1Dist()
    siamese_layer._name = 'distance'
    distances = siamese_layer(embedding(inp_image), embedding(val_image))

    #Classification layer
    classifier = Dense(1, activation='sigmoid')(distances)

    return Model(inputs=[inp_image, val_image], outputs=classifier, name='SiameseNetwork')


In [37]:
siamese_model = make_siamese()
siamese_model.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_image (InputLayer)       [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 val_image (InputLayer)         [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 embedding (Functional)         (None, 4096)         38960448    ['input_image[0][0]',            
                                                                  'val_image[0][0]'] 

## Training the siamese model

In [38]:
siamese_model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')

In [41]:
history = siamese_model.fit(train_data, batch_size=16, epochs=2)

Epoch 1/2
27/27 [==============================] - 594s 19s/step - loss: 0.3679 - accuracy: 0.8214
Epoch 2/2
27/27 [==============================] - 534s 20s/step - loss: 0.1561 - accuracy: 0.9476


## Evaluating the model

In [45]:
from tensorflow.keras.metrics import Precision, Recall

In [74]:
tst_it = test_data.as_numpy_iterator()

In [75]:
yL, yhatL = [], []
for i in trange(len(test_data)):
    batch = tst_it.next()
    y_hat = (siamese_model.predict(batch[0])).reshape(1,-1)[0]
    y_hat = np.where(y_hat<0.5, 0, 1)
    y = batch[1]
    yL = yL + list(y)
    yhatL = yhatL + list(y_hat)

  0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
anchor_tst, val_tst, label = tst_it.next()

In [46]:
r = Recall()
p = Precision()

In [82]:
r.update_state(yL, yhatL)
r.result().numpy()

0.97959185

In [84]:
p.update_state(yL, yhatL)
p.result().numpy()

1.0

## Saving the model

In [85]:
siamese_model.save('siamesemodel.h5')

## Reload model

In [86]:
model = tf.keras.models.load_model('siamesemodel.h5', custom_objects={'L1Dist':L1Dist})

In [92]:
model.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_image (InputLayer)       [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 val_image (InputLayer)         [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 embedding (Functional)         (None, 4096)         38960448    ['input_image[0][0]',            
                                                                  'val_image[0][0]'] 